### Install the required packages

In [ ]:
%pip install python-dotenv
%pip install requests
%pip install openai --user
%pip install pip-system-certs

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI
import sys
import os
sys.path.append('../common')
from helper import Helper

api_version = "2024-06-01"

# Load the environment variables
env = os.getenv('GITHUB_ENV') or 'local'
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-QA'

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")
default_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    'x-kpmg-charge-code': '803000015283',
    "x-kpmg-region-override": region_override,
}

helper = Helper(environment, "genai/azure/deployments", api_key, region_override)

helper.log('GITHUB_ENV: ' + env)
helper.log('ENV_NAME: ' + env_name)

In [ ]:
env_config = helper.read_json_config(env_name)

# Check if deployment_info is found
assert env_config is not None, f"Deployment information not found for environment: {env_name}"

chat_model_deployments_list = env_config["api_configuration"]["azure_openai_completions"]['chat_models']
embeddings_model_deployments_list = env_config["api_configuration"]["azure_openai_completions"]['embeddings_model']


helper.log("Chat Models:")
helper.log(chat_model_deployments_list)
helper.log("Embedding Models:")
helper.log(embeddings_model_deployments_list)

### Chat Completion with max_token & temperature

In [ ]:
helper.log("Chat Completion Test")

deployment_id = [x for x in chat_model_deployments_list if "gpt-4o" in x][0]

helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)

# Create the chat completion request
chat_completion = oai_client.chat.completions.create(
    model=deployment_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What's is Python?"}
    ],
    max_tokens=10,
    temperature=0.7
)

# Output the response
#helper.log(chat_completion.choices[0].message.content)
helper.log(chat_completion.choices[0])

# Assert the response
assert len(chat_completion.choices) == 1
assert chat_completion.choices[0].finish_reason=='length'
print(chat_completion)

##### Chat Completion in Json format

In [ ]:
helper.log("Chat Completion in JSON format Test")

import json

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)

# Create the chat completion request
chat_completion = oai_client.chat.completions.create(
    model=deployment_id,
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ]
)

# Output the response
helper.log(chat_completion.choices[0].message.content)

# Assert the response
response_content = json.loads(chat_completion.choices[0].message.content)
assert isinstance(response_content, dict)
assert len(chat_completion.choices) > 0

#### Chat Completions - Function Calling

In [ ]:
helper.log("Chat Completion function calling Test")

import json
from datetime import datetime

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)

# Define the function for simulates a tool
def calculate(operation, a, b):
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    else:
        return "Operation not supported"


# Define the function for the model
functions = [
    {
        "name": "calculate",
        "description": "Performs basic arithmetic operations.",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract"],
                    "description": "operation to perform"
                },
                "a": {
                    "type": "number",
                    "description": "The first number"
                },
                "b": {
                    "type": "number",
                    "description": "The second number"
                }
            },
            "required": ["operation", "a", "b"],
        }
    }
]

# Define the messages
response = oai_client.chat.completions.create(
    model=deployment_id,
    messages=[
        {"role": "user", "content": "Please add 10 and 20."},],
    functions=functions,
    function_call={"name": "calculate"},
)

function_name = response.choices[0].message.function_call.name
arguments = response.choices[0].message.function_call.arguments


arguments = json.loads(arguments)
result = calculate(arguments['operation'], arguments["a"], arguments["b"])

# Output the response
helper.log(result)

# Assert the response
assert result == 30

#### Testcase 348296: Negative TC - Invalid Subscription Key

In [ ]:
helper.log("Testcase 348296: Negative TC - Chat Completion Invalid Subscription Key")
helper.log("Deployment: " + deployment_id)

import json

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": '%',
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "What is the capital of France?"}
        ],
        temperature = 1,
        top_p = 0,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234"
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 401

except Exception as e:
    # helper.log(f"An unexpected exception occurred: {e}")
    error = e
    # helper.log(error)
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 401
    assert "Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription" in error.message
    

#### Testcase 348316: Negative TC - To Verify Error in Chat Completion with Invalid Temperature Parameters Values

In [ ]:
helper.log("Testcase 348316: Negative TC To Verify Error in Chat Completion with Invalid Temperature Parameters Values")
helper.log("Deployment: " + deployment_id)

import json

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 3,
        top_p = 0,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234"
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"An unexpected exception occurred: {e}")
    error = e
    # helper.log(error)
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message
    
# helper.log(chat_completion)

#### Testcase 348317: Negative TC - To Verify Error in Chat Completion with Invalid top_p Parameters Values

In [ ]:
helper.log("Testcase 348317: Negative TC - To Verify Error in Chat Completion with Invalid top_p Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 1,
        top_p = 2,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234"
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message


# helper.log(chat_completion)

#### Testcase 348318: Negative TC - To Verify Error in Chat Completion with Invalid Max Token Parameters Values

In [ ]:
import time
helper.log("Testcase 348318: Negative TC - To Verify Error in Chat Completion with Invalid Max Token Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    #time.sleep(60) # Sleep for 60 seconds
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 1,
        top_p = 0,
        stop = "null",
        max_tokens = 150000,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234"
    )

    # Output the response
    #helper.log(f"Response: " + chat_completion)
    helper.log(chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "This model's maximum context length is 128000 tokens" in error.message
    helper.log(error)



#### Testcase 348319: Negative TC - To Verify Error in Chat Completion with Invalid presence_penalty Parameters Values

In [ ]:
helper.log("Testcase 348319: Negative TC - To Verify Error in Chat Completion with Invalid presence_penalty Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 2,
        top_p = 1,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 3,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234"
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message

# helper.log(chat_completion)

#### Testcase 348320: Negative TC - To Verify Error in Chat Completion with Invalid frequency_penalty Parameters Values

In [ ]:
helper.log("Testcase 348320: Negative TC - To Verify Error in Chat Completion with Invalid frequency_penalty Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 2,
        top_p = 1,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 3,
        logit_bias = {},
        user = "user-1234"
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message

# helper.log(chat_completion)

#### Testcase 348324: Negative TC - To Verify Error in Chat Streaming Completion with Invalid Temperature Parameters Values

In [ ]:
helper.log("Testcase 348324: Negative TC To Verify Error in Chat Streaming Completion with Invalid Temperature Parameters Values")
helper.log("Deployment: " + deployment_id)

import json

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 3,
        top_p = 0,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234",
        stream=True
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"An unexpected exception occurred: {e}")
    error = e
    # helper.log(error)
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message
    
# helper.log(chat_completion)

#### Testcase 348325: Negative TC - To Verify Error in Chat Streaming Completion with Invalid top_p Parameters Values

In [ ]:
helper.log("Testcase 348325: Negative TC - To Verify Error in Chat Streaming Completion with Invalid top_p Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 1,
        top_p = 2,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234",
        stream=True
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message


# helper.log(chat_completion)

#### Testcase 348326: Negative TC - To Verify Error in Chat Streaming Completion with Invalid Max Token Parameters Values

In [ ]:
helper.log("Testcase 348326: Negative TC - To Verify Error in Chat Streaming Completion with Invalid Max Token Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 1,
        top_p = 0,
        stop = "null",
        max_tokens = 15000000,
        presence_penalty = 1,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234",
        stream=True
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "This model's maximum context length is 128000 tokens." in error.message

    #helper.log(chat_completion)
    #helper.log(error)


#### Testcase 348327: Negative TC - To Verify Error in Chat Streaming Completion with Invalid presence_penalty Parameters Values

In [ ]:
helper.log("Testcase 348327: Negative TC - To Verify Error in Chat Streaming Completion with Invalid presence_penalty Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 2,
        top_p = 1,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 3,
        frequency_penalty = 1,
        logit_bias = {},
        user = "user-1234",
        stream=True
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message

# helper.log(chat_completion)

#### Testcase 348328: Negative TC - To Verify Error in Chat Streaming Completion with Invalid frequency_penalty Parameters Values

In [ ]:
helper.log("Testcase 348328: Negative TC - To Verify Error in Chat Streaming Completion with Invalid frequency_penalty Parameters Values")
helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)
try:
    # Create the chat completion request
    chat_completion = oai_client.chat.completions.create(
        model=deployment_id,
        messages=[
            {"role": "user", "content": "Who is PM of India?"}
        ],
        temperature = 2,
        top_p = 1,
        stop = "null",
        max_tokens = 10,
        presence_penalty = 1,
        frequency_penalty = 3,
        logit_bias = {},
        user = "user-1234",
        stream=True
    )

    # Output the response
    helper.log("Response: " + chat_completion)
    helper.log(f"Response status code: {chat_completion.status_code}")
    assert chat_completion.status_code == 400

except Exception as e:
    # helper.log(f"Bad Request error occurred: {e}")
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    assert "Body of the request does not conform to the definition which is associated with the content type application/json" in error.message

# helper.log(chat_completion)

#### Testcase 348339: Negative TC - To Verify Error Embeddings with Invalid encoding_format Values

In [ ]:
helper.log("Testcase 348339: Negative TC - To Verify Error Embeddings with Invalid encoding_format Values")

# Identify the emebedding model
if(type(embeddings_model_deployments_list)=='list'):
    embedding_deployment_id = [x for x in embeddings_model_deployments_list if "embedding" in x]

embedding_deployment_id=embeddings_model_deployments_list

helper.log("Deployment: " + embedding_deployment_id)

try:
    # Create the client
    oai_client = AzureOpenAI(
        azure_endpoint=F"https://{environment}/genai/azure",
        api_version=api_version,
        api_key=api_key,
        default_headers = {
            "Ocp-Apim-Subscription-Key": api_key,
            'x-kpmg-charge-code': '803000015283',
            "x-kpmg-region-override": region_override,
        }
    )

    # Create the embedding request
    response = oai_client.embeddings.create(
        input = "give me list of top 5 It companies",
        model = embedding_deployment_id,
        encoding_format = "int"
    )

    # Output the response
    helper.log("Response: " + response)
    helper.log(f"Response status code: {response.status_code}")
    # Assert the response is invalid
    assert response.status_code == 400
    
except Exception as e:
    error = e
    helper.log(f"Error message: {error.message}, Error Status Code: {error.status_code}")
    assert error.status_code == 400
    #assert "Invalid value for 'encoding_format'" in error.message

#### Testcase 348341: Negative TC To Verify The number of dimensions the resulting output EMBEDDINGS should have. Only supported in `text-embedding-3` and later models with Invalid dimensions value

In [ ]:
helper.log("Testcase 348341: Negative TC To Verify The number of dimensions the resulting output EMBEDDINGS should have. Only supported in 'text-embedding-3' and later models with Invalid dimensions value")

helper.log("Deployment: " + embedding_deployment_id)

try:
    # Create the client
    oai_client = AzureOpenAI(
        azure_endpoint=F"https://{environment}/genai/azure",
        api_version=api_version,
        api_key=api_key,
        default_headers = {
            "Ocp-Apim-Subscription-Key": api_key,
            'x-kpmg-charge-code': '803000015283',
            "x-kpmg-region-override": region_override,
        }
    )

    # Create the embedding request
    response = oai_client.embeddings.create(
        input = "give me list of top 5 It companies",
        model = embedding_deployment_id,
        encoding_format = "float",
        dimensions = 50000
    )

    

except Exception as e:
    error = e
    helper.log(f"Error message: {str(error)}")
    assert "Error code: 400" in str(error)

    if "ada" in embedding_deployment_id:
        assert "This model does not support specifying dimensions" in str(error)
    else:
        assert "Invalid value for 'dimensions'" in str(error)